In [73]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import krippendorff
import os
import seaborn as sns
import numpy as np
from collections import Counter
import ast

In [53]:
#df = pd.read_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/annotation_output/annotated300.jsonl", orient="records", lines=True)

In [54]:
stem = "/shared/3/projects/benlitterer/podcastData/annotation/label1000/host_guest_annotation/annotation_output/"
toIgnore = []
folderNames = os.listdir(stem)
folderNames = [item for item in folderNames if "." not in item and item != "archived_users"]

In [55]:

#we actually need to concatenate a bunch of files to get who annotated what  
dfList = []

for fName in folderNames: 
    fPath = stem + fName + "/annotated_instances.jsonl" 
    try: 
        currDf = pd.read_json(fPath, orient="records", lines=True)
        currDf["annotatorId"] = fName
        dfList.append(currDf)
    except: 
        pass
df = pd.concat(dfList)

dictList = list(df["label_annotations"])
labels = [int(list(item["sentiment"].values())[0]) for item in dictList]
df["annotation"] = labels

/var/tmp/ipykernel_2428918/3674613519.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  currDf = pd.read_json(fPath, orient="records", lines=True)


In [56]:
df.shape

(3223, 7)

In [57]:
#df = df.explode(["label_annotations"]).reset_index(drop=True)
dictList = list(df["label_annotations"])
labels = [int(list(item["sentiment"].values())[0]) for item in dictList]
df["annotation"] = labels

In [58]:
#extract "check" rows
idList = ["https://anchor.fm/s/7f12654/podcast/play/14320049/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2020-4-26%2F76850511-44100-2-e778a17d843dd.m4a",
"https://anchor.fm/s/11028184/podcast/play/14212379/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fproduction%2F2020-4-24%2F76170728-44100-2-109d447b04488.mp3",
"https://anchor.fm/s/24044d44/podcast/play/14503148/https%3A%2F%2Fd3ctxlq1ktw2nl.cloudfront.net%2Fstaging%2F2020-4-30%2F78020478-44100-1-1e08e64348a62916.mp3"]

def isCheck(inStr): 
    if "CHECK" in inStr:
        return True
    return False 

checkDf = df[df["id"].apply(isCheck)]


In [59]:
checkDf.shape

(225, 7)

In [60]:
#do we first need to find missing rows and fill them in? 
nonCheckDf = df[df["id"].apply(isCheck) == False]
nonCheckDf.head() 

,id,displayed_text,label_annotations,span_annotations,behavioral_data,annotatorId,annotation
0,https://anchor.fm/s/1e830af4/podcast/play/1294...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'host': '1'}},{},{'time_string': 'Time spent: 0d 0h 0m 14s '},663ea7b9da97cba4d63a70ef,1
1,https://traffic.libsyn.com/secure/onthebluelin...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'host': '1'}},{},{'time_string': 'Time spent: 0d 0h 0m 6s '},663ea7b9da97cba4d63a70ef,1
2,http://dts.podtrac.com/redirect.mp3/static1.sq...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'guest': '2'}},{},{'time_string': 'Time spent: 0d 0h 0m 34s '},663ea7b9da97cba4d63a70ef,2
3,https://archive.org/download/vuren-special-1-f...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'neither': '3'}},{},{'time_string': 'Time spent: 0d 0h 1m 13s '},663ea7b9da97cba4d63a70ef,3
4,https://traffic.libsyn.com/secure/tappingqanda...,<span style='background-color:#00FF00'>Target ...,{'sentiment': {'host': '1'}},{},{'time_string': 'Time spent: 0d 0h 2m 49s '},663ea7b9da97cba4d63a70ef,1


In [61]:
nonCheckDf.shape

(2998, 7)

In [62]:
nonCheckDf = df[df["id"].apply(isCheck) == False]
nonCheckDf.shape
nonCheckDf = nonCheckDf[["annotatorId", "annotation", "id"]].sort_values("annotatorId").groupby("id").agg(list)
nonCheckDf["annotStr"] = nonCheckDf["annotatorId"].apply(lambda x: ",".join(x))

In [63]:
#get unique, full teams of annotators 
TEAM_SIZE=3
fullAnnotTeams = [item for item in set(nonCheckDf["annotStr"]) if len(item.split(",")) == TEAM_SIZE]
fullAnnotTeams = [item.split(",") for item in fullAnnotTeams]

#this takes any annotator Id 
#and grabs the team that they were a part of 
def getFullAnnotList(inStr): 
    for annotTeam in fullAnnotTeams: 
        if inStr in annotTeam: 
            return annotTeam 

In [64]:
#look at incomplete rows 
incompleteRows = nonCheckDf[nonCheckDf["annotation"].apply(len) < 3]
incompleteIds = list(incompleteRows.index)

print(incompleteRows.iloc[0])
print(incompleteRows.iloc[1])
#get full annotation teams we should have
print(getFullAnnotList(incompleteRows.iloc[0]["annotatorId"][0]))
print(getFullAnnotList(incompleteRows.iloc[1]["annotatorId"][0]))

annotatorId    [663ea7b9da97cba4d63a70ef, 663eba71ccdbc7e24a7...
annotation                                                [3, 3]
annotStr       663ea7b9da97cba4d63a70ef,663eba71ccdbc7e24a72ad73
Name: https://archive.org/download/vuren-special-1-final/VurenSpecial1Final.mp3, dtype: object
annotatorId    [66316beef88431620ea43fee, 663ea133a2b25b50d10...
annotation                                                [3, 2]
annotStr       66316beef88431620ea43fee,663ea133a2b25b50d1078fc2
Name: https://traffic.omny.fm/d/clips/43d4db60-81e7-4097-aef1-a8b300fb00dd/f23698d1-f145-4dae-9e31-a96500e507cd/9b8f7c43-d721-42af-85b0-abbf00eb5ca3/audio.mp3?utm_source=Podcast&in_playlist=c83c2bea-1597-442c-9fcf-a96500e507cd&t=1589811505, dtype: object
['65a84bcfd24dfc3650843923', '663ea7b9da97cba4d63a70ef', '663eba71ccdbc7e24a72ad73']
['5cd0a1a291c2aa00010c7c92', '66316beef88431620ea43fee', '663ea133a2b25b50d1078fc2']


In [65]:


def fillMissingAnnot(inRow): 
    annotStr = inRow["annotStr"]
    if len(annotStr.split(",")) == TEAM_SIZE: 
        return inRow
    else: 
        #get the full team of annotators we should have, and fill in 
        #our actual annotations so we have a full set in the right order 
        currAnnots = inRow["annotatorId"]
        currAnnotations = inRow["annotation"]
        fAnnot = annotStr.split(",")[0]
        fTeam = getFullAnnotList(fAnnot)
        outRow = [[],[],""]
        annotIter = 0 
        print(fTeam)
        for i, annotator in enumerate(fTeam): 

            outRow[0].append(annotator)
            if i > 0: 
                outRow[2] += f",{annotator}"
            else: 
                outRow[2] += annotator
            
            #if we arrive at a missing annotation 
            if annotator not in currAnnots: 
                outRow[1].append(np.nan)
            else: 
                outRow[1].append(currAnnotations[annotIter])
                annotIter += 1
        
        return outRow

#this is the filled-in version of nonCheckDf
nonCheckDfImp = nonCheckDf.apply(fillMissingAnnot, axis=1)



['65a84bcfd24dfc3650843923', '663ea7b9da97cba4d63a70ef', '663eba71ccdbc7e24a72ad73']
['5cd0a1a291c2aa00010c7c92', '66316beef88431620ea43fee', '663ea133a2b25b50d1078fc2']


,annotatorId,annotation,annotStr
id,,,
http://awscdn.podcasts.com/Episode-5-Interview-with-Andrea-Bailey-f9f8.mp3,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[1, 1, 1]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56..."
http://awscdn.podcasts.com/What-If-Aang-Hadnt-Disappeared-Right-Before-Sozins-Comet-c0b2.mp3,"[6461e54ee3ce9d9554deda21, 65c17f809c893363dae...","[2, 1, 1]","6461e54ee3ce9d9554deda21,65c17f809c893363dae6b..."
http://bnspodcast.blob.core.windows.net/podcasts/HB_2020_06_02.mp3,"[5efb5360f3f69c084c1ca850, 6151b3079d837c7cd5c...","[3, 3, 3]","5efb5360f3f69c084c1ca850,6151b3079d837c7cd5c80..."
http://cast.rocks/hosting/12797/The-Real-Story-on-Gout.mp3,"[5b6364e153350a0001c0402a, 5d9d5debf3462400144...","[1, 1, 1]","5b6364e153350a0001c0402a,5d9d5debf346240014428..."
http://dts.podtrac.com/redirect.mp3/feeds.soundcloud.com/stream/820232152-humansofhealthcare-021-joseph-thompson-failure-and-resilience.mp3,"[65dd1989214a774c36a0854a, 663b65caecf1e42556b...","[1, 1, 1]","65dd1989214a774c36a0854a,663b65caecf1e42556ba2..."


In [74]:
nonCheckDfImp.head() 

,annotatorId,annotation,annotStr
id,,,
http://awscdn.podcasts.com/Episode-5-Interview-with-Andrea-Bailey-f9f8.mp3,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[1, 1, 1]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56..."
http://awscdn.podcasts.com/What-If-Aang-Hadnt-Disappeared-Right-Before-Sozins-Comet-c0b2.mp3,"[6461e54ee3ce9d9554deda21, 65c17f809c893363dae...","[2, 1, 1]","6461e54ee3ce9d9554deda21,65c17f809c893363dae6b..."
http://bnspodcast.blob.core.windows.net/podcasts/HB_2020_06_02.mp3,"[5efb5360f3f69c084c1ca850, 6151b3079d837c7cd5c...","[3, 3, 3]","5efb5360f3f69c084c1ca850,6151b3079d837c7cd5c80..."
http://cast.rocks/hosting/12797/The-Real-Story-on-Gout.mp3,"[5b6364e153350a0001c0402a, 5d9d5debf3462400144...","[1, 1, 1]","5b6364e153350a0001c0402a,5d9d5debf346240014428..."
http://dts.podtrac.com/redirect.mp3/feeds.soundcloud.com/stream/820232152-humansofhealthcare-021-joseph-thompson-failure-and-resilience.mp3,"[65dd1989214a774c36a0854a, 663b65caecf1e42556b...","[1, 1, 1]","65dd1989214a774c36a0854a,663b65caecf1e42556ba2..."


In [75]:
outStem = "/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/inputs/"

#write our full dataframe of MACE inputs so that we can merge things back in when we need
nonCheckDfImp.reset_index().to_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/inputsDf.jsonl", orient="records", lines=True)

for annotStr in nonCheckDfImp["annotStr"].unique(): 
    currDf = nonCheckDfImp[nonCheckDfImp["annotStr"] == annotStr]
    currAnnotDf = pd.DataFrame(currDf["annotation"].tolist(), dtype="Int64") #.fillna("")
    if len(currAnnotDf.dropna()) != len(currAnnotDf): 
        print(annotStr)
        print(currAnnotDf)

    currAnnotDf.to_csv(f"{outStem}{annotStr}.csv", sep=",", index=False, header=False)

65a84bcfd24dfc3650843923,663ea7b9da97cba4d63a70ef,663eba71ccdbc7e24a72ad73
       0  1  2
0      3  2  2
1      3  3  3
2      2  2  2
3      3  3  3
4      1  1  1
5      2  2  2
6      1  1  1
7      2  2  2
8      2  1  2
9      1  1  1
10     1  1  1
11     2  2  2
12     1  1  1
13  <NA>  3  3
14     1  1  3
15     1  1  1
16     3  2  2
17     1  1  1
18     3  3  3
19     1  1  1
20     3  3  3
21     2  3  3
22     2  2  2
23     1  1  1
24     2  2  2
25     3  3  3
26     3  1  1
27     3  3  3
28     2  2  1
29     1  1  1
30     1  1  1
31     1  1  1
32     3  3  3
33     3  3  3
34     2  2  2
35     3  3  2
36     2  2  2
37     2  2  2
38     1  1  1
39     2  1  1
5cd0a1a291c2aa00010c7c92,66316beef88431620ea43fee,663ea133a2b25b50d1078fc2
       0  1  2
0      1  1  1
1      3  3  3
2      3  3  3
3      3  3  3
4      3  3  3
5      3  3  3
6      1  3  3
7      2  3  2
8      2  2  2
9      2  2  2
10     2  3  3
11     1  1  1
12     3  2  2
13     3  3  3
14     1  

In [38]:
currAnnotDf

,0,1,2
0,3,3,3
1,1,1,1
2,3,3,1
3,3,3,3
4,2,2,2
5,2,3,2
6,1,3,1
7,3,3,3
8,1,1,1
9,2,2,2
